# Add extra information to DRZ auction results

Query to the open data dataset of the RDW.

It may take a while (10 min) to query all auction results.
- - - - 

### User variables


In [1]:
Date = '2020-02' # yyyy-mm

toggle_to_opbod = True

### Modules and functions

In [2]:
import pandas as pd
import re 
# to keep api key hidden import this from sub dir
import assets.hidden_api_keys as hidden_api_keys
from time import sleep

# base url
apiurl = 'https://opendata.rdw.nl/resource/m9d7-ebf2.json?$$app_token=' + hidden_api_keys.socrata_apptoken + '&'

def get_json_from_api(url,reg,c=0):
    
    '''Get json object from api'''
    
    import time

    c+=1
    try:
        df=pd.read_json(url + 'kenteken=' + reg.replace('-','').upper()).to_dict()
    except:
        if c > 10:
            print(url,reg)
            raise 
        else:
            print('pause 2 sec and try again!')
            time.sleep(2)
            df = get_json_from_api(url,reg,c)
    
    return df
    
# get_json_from_api(apiurl,'61-sf-FG')

### Load auction results

In [3]:
if toggle_to_opbod:
    file_name = '../../../python-nb/data/drz-data-opbod-{}.pkl'.format(Date)
else:
    file_name = '../data/drz-data-{}.pkl'.format(Date)
print(file_name)
drz = pd.read_pickle(file_name)

../../../python-nb/data/drz-data-opbod-2020-02.pkl


### query rdw

In [4]:
# see what lots have a Dutch registration (license number).
hasReg = (~drz.Reg.isnull()) & (drz.Reg != 'onbekend') & (drz.Reg != '') & (~drz.LotType.isin([
    'Vaartuig',
    'Jetski',
    'Sloep',
    'Speedboot',
    'Vaartuig (Type onbekend)',
    'Motorvaartuig met opbouw (Pleziervaartuig)',
    'Aanhangwagen',
]))

# make a copy and add info
rdw = drz.copy()


In [5]:
def get_query_url(api_url, keys, token=hidden_api_keys.socrata_apptoken, field='kenteken'):
    '''construct query url'''
    
    # convert list to string
    id_list = ''.join(["'{}', ".format(k) for k in keys])
    id_list = id_list[0:-2] # remove trailing ', '
    q = api_url + '?$$app_token=' + token + '&$where='
    # add escaped soql
    soql = field + ' in(' + id_list + ')'
    q += urllib.parse.quote(soql)
    return q

Create list of dataframes with different api results

In [6]:
import urllib

# empty dictionary
dfs_regs = dict()
# first element is all registrations
key = 'registations'
dfs_regs[key] = rdw.loc[hasReg,['Reg']]
dfs_regs[key]['kenteken'] = dfs_regs[key].Reg.apply(lambda r: r.replace('-','').upper())
dfs_regs[key].index.name = 'lot_index'
dfs_regs[key] = dfs_regs[key].reset_index().set_index('kenteken')
display(dfs_regs[key].tail())
print(dfs_regs[key].shape)


,lot_index,Reg
kenteken,,
VB543Z,2020-2-8301,VB-543-Z
RR974N,2020-2-8302,RR-974-N
VP557Z,2020-2-8303,VP-557-Z
SXJV05,2020-2-8304,SX-JV-05
94JKL7,2020-2-8306,94-JKL-7


(61, 2)


In [7]:
# do main api first to get other possible apis
api_name = 'api_gekentekende_voertuigen'
key = re.sub('^api_','',api_name)
api_url = 'https://opendata.rdw.nl/resource/m9d7-ebf2.json'
regs = dfs_regs['registations'].Reg.values
regs = [r.replace('-','').upper() for r in regs]
print(len(regs),'registrations in this set')

# query data base
q = get_query_url(api_url,regs)
dfs_regs[key] = pd.read_json(q)
dfs_regs[key].set_index('kenteken', inplace=True)
display(dfs_regs[key].tail())
print(dfs_regs[key].shape)

# query other available apis
for api_name in [c for c in dfs_regs['gekentekende_voertuigen'].columns if c.startswith('api')]:
    print(api_name, end=': ')
    key = re.sub('^api_','',api_name)
    for api_url in dfs_regs['gekentekende_voertuigen'][api_name].unique():
        print(api_url)
        # query the web
        q = get_query_url(api_url,regs)
        df0 = pd.read_json(q)
        # name of index
        df0.columns.name = api_name

        # query should return 'kenteken', make it the index
        if df0.shape[0] != 0:
            df0.set_index('kenteken', inplace=True)
            
        # Some apis return multiple values. Pivot around index number ("volgnummer")
        if api_name == 'api_gekentekende_voertuigen_assen':
            df0 = pd.pivot(df0, columns='as_nummer')

        elif api_name == 'api_gekentekende_voertuigen_brandstof':
            df0 = pd.pivot(df0, columns='brandstof_volgnummer')

        elif api_name == 'api_gekentekende_voertuigen_carrosserie':
            df0 = pd.pivot(df0, columns='carrosserie_volgnummer')

        elif api_name == 'api_gekentekende_voertuigen_carrosserie_specifiek':
            df0 = pd.pivot(df0, columns='carrosserie_volgnummer')

        # squeeze multi index
        one_level = [
            re.sub('^api_gekentekende_voertuigen_','',api_name) + '_' + '_'.join(
                [str(c) if type(c)==int else c for c in l]
            ) for l in df0.columns
        ]
        df0.columns = one_level

        # add to list
        display(df0.tail())
        print(df0.shape)
        dfs_regs[key]=df0

61 registrations in this set


,voertuigsoort,merk,handelsbenaming,vervaldatum_apk,datum_tenaamstelling,bruto_bpm,inrichting,aantal_zitplaatsen,eerste_kleur,tweede_kleur,...,maximum_massa_samenstelling,maximum_trekken_massa_geremd,zuinigheidslabel,typegoedkeuringsnummer,variant,uitvoering,aantal_rolstoelplaatsen,catalogusprijs,type,maximum_ondersteunende_snelheid
kenteken,,,,,,,,,,,,,,,,,,,,,
VDF37K,Bedrijfsauto,PEUGEOT,BIPPER,20200411,20200129,4269.0,gesloten opbouw,2.0,N.v.t.,N.v.t.,...,2750.0,NaN,NaN,e3*2007/46*0012*03,AFHZ0,N/A,0.0,17095.0,A,0.0
VP294X,Bedrijfsauto,MERCEDES-BENZ,SPRINTER,20200122,20200129,11066.0,gesloten opbouw,3.0,N.v.t.,N.v.t.,...,5500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G0315DE37C,NaN
VP557Z,Bedrijfsauto,SSANGYONG,SSANGYONG REXTON,20200305,20200129,13058.0,gesloten opbouw,2.0,N.v.t.,N.v.t.,...,6050.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
XJFL09,Personenauto,VOLKSWAGEN,LUPO,20191113,20200129,1900.0,hatchback,4.0,BLAUW,Niet geregistreerd,...,2000.0,650.0,NaN,e1*97/27*0085*01,SCAHTX01,SGFM5020021N1I4,0.0,NaN,NaN,NaN
ZNNG31,Personenauto,VOLKSWAGEN,NEW BEETLE,20200916,20200129,5705.0,hatchback,4.0,GRIJS,Niet geregistreerd,...,2650.0,1000.0,NaN,e1*97/27*0106*00,SCAQYX0,SGFM52J046M4,0.0,NaN,NaN,NaN


(61, 54)
api_gekentekende_voertuigen_assen: https://opendata.rdw.nl/resource/3huj-srit.json


,assen_aantal_assen_1,assen_aantal_assen_2,assen_plaatscode_as_1,assen_plaatscode_as_2,assen_spoorbreedte_1,assen_spoorbreedte_2,assen_wettelijk_toegestane_maximum_aslast_1,assen_wettelijk_toegestane_maximum_aslast_2,assen_technisch_toegestane_maximum_aslast_1,assen_technisch_toegestane_maximum_aslast_2,assen_hefas_1,assen_hefas_2,assen_aangedreven_as_1,assen_aangedreven_as_2
kenteken,,,,,,,,,,,,,,
VDF37K,2,2,V,A,147,147,900,950,900.0,950.0,N,N,NaN,NaN
VP294X,2,2,V,A,171,173,1650,2250,NaN,NaN,N,N,NaN,NaN
VP557Z,2,2,V,A,155,154,1250,1585,NaN,NaN,N,N,NaN,NaN
XJFL09,2,2,NaN,NaN,139,140,750,690,750.0,690.0,NaN,NaN,NaN,NaN
ZNNG31,2,2,NaN,NaN,152,149,940,800,940.0,800.0,NaN,NaN,NaN,NaN


(61, 14)
api_gekentekende_voertuigen_brandstof: https://opendata.rdw.nl/resource/8ys7-d773.json


...

(61, 2)
api_gekentekende_voertuigen_carrosserie_specifiek: https://opendata.rdw.nl/resource/jhie-znh9.json


,carrosserie_specifiek_carrosserie_voertuig_nummer_code_volgnummer_1,carrosserie_specifiek_carrosseriecode_1,carrosserie_specifiek_carrosserie_voertuig_nummer_europese_omschrijving_1
kenteken,,,
V599JJ,1,3,Gesloten opbouw
VB543Z,1,3,Gesloten opbouw
VDF37K,1,3,Gesloten opbouw
VP294X,1,3,Gesloten opbouw
VP557Z,1,3,Gesloten opbouw


(9, 3)
api_gekentekende_voertuigen_voertuigklasse: https://opendata.rdw.nl/resource/kmfi-hrps.json


""


(0, 0)


In [8]:
# Merge dataframes from different apis
df_regs = pd.concat(dfs_regs.values(), axis='columns', sort=False)
# add timestamp
df_regs['TimeStamp'] = pd.to_datetime('now').strftime('%Y%m%d')
# set lot id as index
df_regs.index.name = 'kenteken'
df_regs = df_regs.reset_index().set_index('lot_index')
display(df_regs.tail())
print(df_regs.shape)

,kenteken,Reg,voertuigsoort,merk,handelsbenaming,vervaldatum_apk,datum_tenaamstelling,bruto_bpm,inrichting,aantal_zitplaatsen,...,brandstof_uitstoot_deeltjes_zwaar_1,brandstof_uitstoot_deeltjes_zwaar_2,brandstof_milieuklasse_eg_goedkeuring_zwaar_1,brandstof_milieuklasse_eg_goedkeuring_zwaar_2,carrosserie_carrosserietype_1,carrosserie_type_carrosserie_europese_omschrijving_1,carrosserie_specifiek_carrosserie_voertuig_nummer_code_volgnummer_1,carrosserie_specifiek_carrosseriecode_1,carrosserie_specifiek_carrosserie_voertuig_nummer_europese_omschrijving_1,TimeStamp
lot_index,,,,,,,,,,,,,,,,,,,,,
2020-2-8301,VB543Z,VB-543-Z,Bedrijfsauto,VOLKSWAGEN,TOUAREG,20200117,20200129,NaN,gesloten opbouw,2.0,...,NaN,NaN,NaN,NaN,BB,Bestelwagen,1.0,3.0,Gesloten opbouw,20200207
2020-2-8302,RR974N,RR-974-N,Personenauto,AUDI,AUDI A3,20190531,20190529,409.0,stationwagen,5.0,...,NaN,NaN,NaN,NaN,AC,Stationwagen,NaN,NaN,NaN,20200207
2020-2-8303,VP557Z,VP-557-Z,Bedrijfsauto,SSANGYONG,SSANGYONG REXTON,20200305,20200129,13058.0,gesloten opbouw,2.0,...,NaN,NaN,NaN,NaN,BB,Bestelwagen,1.0,3.0,Gesloten opbouw,20200207
2020-2-8304,SXJV05,SX-JV-05,Personenauto,CITROEN,S0HDZF,20191129,20200129,1960.0,hatchback,5.0,...,NaN,NaN,NaN,NaN,AB,Hatchback,NaN,NaN,NaN,20200207
2020-2-8306,94JKL7,94-JKL-7,Personenauto,AUDI,AUDI A6,20200822,20200129,18597.0,stationwagen,5.0,...,NaN,NaN,NaN,NaN,AC,Stationwagen,NaN,NaN,NaN,20200207


(61, 106)


In [9]:
# empty dict
dfs_confcodes = dict()
# Conformity codes have sub-divisions. Four fields make a super key
key = 'conformity_codes'
dfs_confcodes[key] = dfs_regs['gekentekende_voertuigen'][[
    'typegoedkeuringsnummer', 
    'uitvoering', 
    'variant', 
    'volgnummer_wijziging_eu_typegoedkeuring'
]].dropna().drop_duplicates()
dfs_confcodes[key].reset_index(drop=True, inplace=True)
print(len(dfs_confcodes[key]),'conformity codes in this set')

display(dfs_confcodes[key].tail())
print(dfs_confcodes[key].shape)

36 conformity codes in this set


,typegoedkeuringsnummer,uitvoering,variant,volgnummer_wijziging_eu_typegoedkeuring
31,e2*93/81*0205*00,KD2,C,0.0
32,e2*2007/46*0014*13,FLA0AE,FLA0,0.0
33,e3*2007/46*0012*03,N/A,AFHZ0,0.0
34,e1*97/27*0085*01,SGFM5020021N1I4,SCAHTX01,1.0
35,e1*97/27*0106*00,SGFM52J046M4,SCAQYX0,0.0


(36, 4)


In [10]:
# do conformity api and again get other possible apis
api_name = 'api_eeg_voertuigtypegoedkeuring'
key = re.sub('^api_','',api_name)
api_url = 'https://opendata.rdw.nl/resource/55kv-xf7m.json'

# query data base
# will not use sub-division, but long (year with century) version of conformity code
q = get_query_url(api_url, 
                  dfs_confcodes['conformity_codes'].typegoedkeuringsnummer.unique(), 
                  field='typegoedkeuringsnummer')
dfs_confcodes[key] = pd.read_json(q)
# more than one conformity code?
assert not (dfs_confcodes[key].groupby('typegoedkeuringsnummer')['typegoedkeuringsnummer'].count() > 1).any()
dfs_confcodes[key].set_index('typegoedkeuringsnummer', inplace=True)
display(dfs_confcodes[key].tail())
print(dfs_confcodes[key].shape)

,eu_type_goedkeuringssleutel,landcode_eeg_typegoedkeuring,richtlijn_nr_laatste_wijziging,eeg_basis_goedkeuringsnummer,eeg_uitbreiding_goedkeuringsnummer,eeg_typegoedkeuringsdatum,europese_typegoedkeurings_registratie_datum,eeg_ece_voertuig_categorie_bij_type,fabrikant,europese_typegoedkeuring_status,...,api_carrosserie_uitvoering_nummerieke_code,api_handelsbenaming_uitvoering,api_merk_uitvoering_toegestaan,api_motor_uitvoering,api_motor_uitvoering_brandstof,api_plaatsaanduiding_uitvoering,api_subcategorie_uitvoering,api_uitvoeringverbruik_per_uitgave,api_versnellingsbak_uitvoering,type_fabrikant
typegoedkeuringsnummer,,,,,,,,,,,,,,,,,,,,,
e3*2001/116*0217*22,e3*01/116*0217*22,e3,01/116,217,22,20101214,20101214,M1,FCA ITALY SPA,BT,...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,199
e3*2007/46*0012*03,e3*07/46*0012*03,e3,07/46,12,3,20110727,20110727,N1,AUTOMOBILES PEUGEOT,BT,...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,A
e4*2001/116*0085*02,e4*01/116*0085*02,e4,01/116,85,2,20050826,20050826,M1,KIA MOTORS CORPORATION,BT,...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,NaN
e4*2001/116*0092*02,e4*01/116*0092*02,e4,01/116,92,2,20060130,20060130,M1,GM DAEWOO AUTO & TECHNOLOGY COMPANY,BT,...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,NaN
e9*98/14*0026*09,e9*98/14*0026*09,e9,98/14,26,9,20000720,20000720,M1,SEAT S.A.,BT,...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,NaN


(36, 26)


In [11]:
# add slightly different keys (year has no century)
dfs_confcodes['conformity_codes'] = dfs_confcodes['conformity_codes'].merge(
    dfs_confcodes['eeg_voertuigtypegoedkeuring'].eu_type_goedkeuringssleutel, 
    how='left', 
    left_on='typegoedkeuringsnummer', 
    right_index=True
)
dfs_confcodes['conformity_codes'].tail()

,typegoedkeuringsnummer,uitvoering,variant,volgnummer_wijziging_eu_typegoedkeuring,eu_type_goedkeuringssleutel
31,e2*93/81*0205*00,KD2,C,0.0,e2*93/81*0205*00
32,e2*2007/46*0014*13,FLA0AE,FLA0,0.0,e2*07/46*0014*13
33,e3*2007/46*0012*03,N/A,AFHZ0,0.0,e3*07/46*0012*03
34,e1*97/27*0085*01,SGFM5020021N1I4,SCAHTX01,1.0,e1*97/27*0085*01
35,e1*97/27*0106*00,SGFM52J046M4,SCAQYX0,0.0,e1*97/27*0106*00


In [ ]:
# query other available apis
for api_name in [c for c in dfs_confcodes['eeg_voertuigtypegoedkeuring'].columns if c.startswith('api')]:
    print(api_name, end=': ')
    key = re.sub('^api_','',api_name)
    for api_url in dfs_confcodes['eeg_voertuigtypegoedkeuring'][api_name].unique():
        
        # reformat url
        M=re.search('https://opendata.rdw.nl/.*/([a-z0-9]{4}-[a-z0-9]{4})$', api_url)
        api_url = 'https://opendata.rdw.nl/resource/{}.json'.format(M[1])
        print(api_url)

#         if api_name in [
#             'api_as_gegevens_eeg_uitvoering',
#             'api_basisgegevens_eeg_uitvoering',
#             'api_carrosserie_uitvoering',
#             'api_carrosserie_uitvoering_klasse',
#             'api_carrosserie_uitvoering_nummerieke_code',            
#             'api_handelsbenaming_uitvoering',
#             'api_merk_uitvoering_toegestaan',
#             'api_motor_uitvoering',
#             'api_motor_uitvoering_brandstof',
#             'api_plaatsaanduiding_uitvoering',
#             'api_subcategorie_uitvoering',
#             'api_uitvoeringverbruik_per_uitgave',
#             'api_versnellingsbak_uitvoering',
#         ]: continue
        
        # query the web
        # unfortunately this needs to be done one by one, because conformity code is not unique                   
        df0 = pd.DataFrame()
        for ix, row in dfs_confcodes['conformity_codes'].iterrows():
            q = api_url
            q += '?$$app_token=' + hidden_api_keys.socrata_apptoken 
            q += '&{}=\'{}\''.format('eu_type_goedkeuringssleutel', urllib.parse.quote(row.eu_type_goedkeuringssleutel))
            q += '&{}=\'{}\''.format('eeg_uitvoeringscode', urllib.parse.quote(row.uitvoering))
            q += '&{}=\'{}\''.format('eeg_variantcode', urllib.parse.quote(row.variant))
            q += '&{}={:.0f}'.format('uitvoering_wijzigingsnummer', row.volgnummer_wijziging_eu_typegoedkeuring)
                
            n_try = 0
            OK = False
            while (n_try < 10) & (not OK):
                try:
                    res = pd.read_json(q)
                    OK = True
                    n_try = 0
                except:
                    n_try +=1
                    sleep(10)
                if n_try == 10:
                    raise

            if len(res) == 0:
                continue

            # matching data type with 'codes' for merging
            res.eu_type_goedkeuringssleutel = res.eu_type_goedkeuringssleutel.astype(str)
            res.eeg_uitvoeringscode = res.eeg_uitvoeringscode.astype(str)
            res.eeg_variantcode = res.eeg_variantcode.astype(str)
            res.uitvoering_wijzigingsnummer = res.uitvoering_wijzigingsnummer.astype(float)
            res.set_index([
                'eu_type_goedkeuringssleutel', 
                'eeg_uitvoeringscode', 
                'eeg_variantcode', 
                'uitvoering_wijzigingsnummer'
            ], inplace=True)
                
            if api_name == 'api_as_gegevens_eeg_uitvoering':               
                piv = res.pivot(columns='asnummer')
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)                
            elif api_name == 'api_handelsbenaming_uitvoering':                
                piv = res.pivot(columns='volgnummer')
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)
            elif api_name == 'api_carrosserie_uitvoering':                
                piv = res.pivot(columns='carrosserie_volgnummer')
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)
            elif api_name == 'api_plaatsaanduiding_uitvoering':
                piv = res.pivot(columns='plaats_aanduiding_volgnummer')
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)            
            elif api_name == 'api_uitvoeringverbruik_per_uitgave':
                piv = res.pivot(columns='uitvgavenummer_verbruikboek')
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)
            elif api_name == 'api_motor_uitvoering':
                piv = res.pivot(columns='volgnummer')
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)
            elif api_name == 'api_versnellingsbak_uitvoering':
                piv = res.pivot(columns='volgnummer')
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)
            elif api_name == 'api_motor_uitvoering_brandstof':
                piv = pd.pivot_table(res, index=res.index.names, columns=['volgnummer', 'brandstof_volgnummer'])
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)                
            elif api_name == 'api_merk_uitvoering_toegestaan':
                piv = pd.pivot_table(res, index=res.index.names, values=res.columns, aggfunc=list)
                df0 = pd.concat([df0, piv], sort=False)
            else:
                assert res.shape[0] == 1
                columns = ['{}_{}'.format(re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)), c) for c in res.columns] 
                res.columns = columns
                df0 = pd.concat([df0, res], sort=False)
                
            print('.', end='')
        if len(df0)==0:
            print('No results for this api')
            continue


        # add to dict
        dfs_confcodes[key] = df0
        display(dfs_confcodes[key].tail())
        print(dfs_confcodes[key].shape)


In [13]:
# Merge dataframes from different apis

# first merge first two results
tmp = dfs_confcodes['conformity_codes'].merge(dfs_confcodes['eeg_voertuigtypegoedkeuring'], how='left', 
                                              left_on='typegoedkeuringsnummer',
                                              right_index=True
                                             )
tmp['uitvoering']
tmp.rename(columns={
    'eu_type_goedkeuringssleutel_x': 'eu_type_goedkeuringssleutel',
    'uitvoering': 'eeg_uitvoeringscode',
    'variant': 'eeg_variantcode',
    'volgnummer_wijziging_eu_typegoedkeuring': 'uitvoering_wijzigingsnummer',
}, inplace=True)
tmp.set_index(['eu_type_goedkeuringssleutel', 'eeg_uitvoeringscode', 'eeg_variantcode', 'uitvoering_wijzigingsnummer'], inplace=True)

# merge with subsequent api results
df_confcodes = pd.concat([tmp] + list(dfs_confcodes.values())[2:], axis='columns', sort=False)
# add timestamp
df_confcodes['TimeStamp'] = pd.to_datetime('now').strftime('%Y%m%d')

display(df_confcodes.tail())
print(df_confcodes.shape)

,,,,typegoedkeuringsnummer,eu_type_goedkeuringssleutel_y,landcode_eeg_typegoedkeuring,richtlijn_nr_laatste_wijziging,eeg_basis_goedkeuringsnummer,eeg_uitbreiding_goedkeuringsnummer,eeg_typegoedkeuringsdatum,europese_typegoedkeurings_registratie_datum,eeg_ece_voertuig_categorie_bij_type,fabrikant,...,uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_17,uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_18,uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_1,uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_2,uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_3,uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_19,versnellingsbak_uitvoering_type_versnellingsbak_1,versnellingsbak_uitvoering_aantal_versnellingen_ondergrens_1,versnellingsbak_uitvoering_aantal_versnellingen_bovengrens_1,TimeStamp
eu_type_goedkeuringssleutel,eeg_uitvoeringscode,eeg_variantcode,uitvoering_wijzigingsnummer,,,,,,,,,,,,,,,,,,,,,
e3*01/116*0217*22,48B,AXY1A,0.0,e3*2001/116*0217*22,e3*01/116*0217*22,e3,01/116,217.0,22.0,20101214.0,20101214.0,M1,FCA ITALY SPA,...,D,D,NaN,NaN,NaN,D,H,5.0,5.0,20200207
e3*07/46*0012*03,N/A,AFHZ0,0.0,e3*2007/46*0012*03,e3*07/46*0012*03,e3,07/46,12.0,3.0,20110727.0,20110727.0,N1,AUTOMOBILES PEUGEOT,...,NaN,NaN,NaN,NaN,NaN,NaN,H,5.0,5.0,20200207
e4*01/116*0085*02,521111,M211,0.0,e4*2001/116*0085*02,e4*01/116*0085*02,e4,01/116,85.0,2.0,20050826.0,20050826.0,M1,KIA MOTORS CORPORATION,...,E,E,NaN,NaN,NaN,NaN,H,5.0,5.0,20200207
e4*01/116*0092*02,111,KH1,0.0,e4*2001/116*0092*02,e4*01/116*0092*02,e4,01/116,92.0,2.0,20060130.0,20060130.0,M1,GM DAEWOO AUTO & TECHNOLOGY COMPANY,...,E,F,NaN,NaN,NaN,F,H,5.0,5.0,20200207
e9*98/14*0026*09,SGFM52J0471M1N1,ABAPGX01,1.0,e9*98/14*0026*09,e9*98/14*0026*09,e9,98/14,26.0,9.0,20000720.0,20000720.0,M1,SEAT S.A.,...,G,G,D,D,D,G,H,5.0,5.0,20200207


(37, 218)


In [14]:
# Merge confirmation codes with registrations
df_regs.index.name = 'lot_index'
df = df_regs.reset_index().merge(df_confcodes.reset_index(), how='left',
                   left_on=['typegoedkeuringsnummer', 'uitvoering', 'variant', 'volgnummer_wijziging_eu_typegoedkeuring'],
                   right_on=['typegoedkeuringsnummer', 'eeg_uitvoeringscode', 'eeg_variantcode', 'uitvoering_wijzigingsnummer'],
).set_index('lot_index')
display(df.tail(10))
print(df.shape)
assert all(df.columns.value_counts() == 1)

,kenteken,Reg,voertuigsoort,merk,handelsbenaming,vervaldatum_apk,datum_tenaamstelling,bruto_bpm,inrichting,aantal_zitplaatsen,...,uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_17,uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_18,uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_1,uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_2,uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_3,uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_19,versnellingsbak_uitvoering_type_versnellingsbak_1,versnellingsbak_uitvoering_aantal_versnellingen_ondergrens_1,versnellingsbak_uitvoering_aantal_versnellingen_bovengrens_1,TimeStamp_y
lot_index,,,,,,,,,,,,,,,,,,,,,
2020-2-1106,14XLPT,14-XL-PT,Personenauto,PEUGEOT,307,20200528,20200129,6839.0,stationwagen,5.0,...,G,G,NaN,NaN,NaN,G,H,5.0,5.0,20200207
2020-2-1107,34VFNB,34-VF-NB,Bedrijfsauto,PEUGEOT,PARTNER 170C 1.9D 500,20160316,20200129,NaN,gesloten opbouw,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-2-1108,39BLLN,39-BL-LN,Bedrijfsauto,IVECO,40C,20200321,20200129,NaN,gesloten opbouw,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-2-1111,KV252Z,KV-252-Z,Personenauto,MERCEDES-BENZ,A 180 CDI,20200319,20191030,912.0,stationwagen,5.0,...,F,F,NaN,NaN,NaN,F,A,7.0,7.0,20200207
2020-2-1112,91RRVK,91-RR-VK,Personenauto,AUDI,AUDI A3,20200220,20191028,10028.0,hatchback,5.0,...,G,G,NaN,NaN,NaN,NaN,H,6.0,6.0,20200207
2020-2-8301,VB543Z,VB-543-Z,Bedrijfsauto,VOLKSWAGEN,TOUAREG,20200117,20200129,NaN,gesloten opbouw,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-2-8302,RR974N,RR-974-N,Personenauto,AUDI,AUDI A3,20190531,20190529,409.0,stationwagen,5.0,...,D,D,NaN,NaN,NaN,NaN,A,7.0,7.0,20200207
2020-2-8303,VP557Z,VP-557-Z,Bedrijfsauto,SSANGYONG,SSANGYONG REXTON,20200305,20200129,13058.0,gesloten opbouw,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-2-8304,SXJV05,SX-JV-05,Personenauto,CITROEN,S0HDZF,20191129,20200129,1960.0,hatchback,5.0,...,G,G,D,D,D,NaN,H,5.0,5.0,20200207


(61, 327)


# Merge rdw and drz

In [15]:
rdw = pd.concat([rdw, df.add_prefix('rdw_')], axis='columns', sort=False)
# There should be no duplicates in column names
assert all(rdw.columns.value_counts() == 1)
# indices should match
assert rdw.index.isin(drz.index).all() & drz.index.isin(rdw.index).all()

### Saving

In [16]:
if toggle_to_opbod:
    file_name = '../../../python-nb/data/rdw-data-opbod-{}.pkl'.format(Date)
else:
    file_name = '../data/rdw-data-{}.pkl'.format(Date)
print(file_name)
rdw.to_pickle(file_name)

../../../python-nb/data/rdw-data-opbod-2020-02.pkl
